In [1]:
from PIL import Image
import os
from datetime import datetime
import math
import logging
from typing import List
import argparse
from urllib.request import urlretrieve
import open3d as o3d
import numpy as np
import cv2
import torch

import torch.backends.cudnn as cudnn
from torch.optim import Optimizer, SGD
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from datasets import VideoDatasetEval, VideoDatasetTrainSup
from utils import AverageMeter, setup_logger
from deeplab.network import deeplabv3plus_resnet50
from confusion_matrix import ConfusionMatrix
from model3D.model_utils import *

from plyfile import PlyData
from MinkowskiEngine import SparseTensor

In [2]:
#from dataset3D.dataloader import *
from dataset3D.dataset import ExampleDataset
import torch.optim as optim

In [3]:
#file_name = '/home/dyf/dataset/custom_ycb/0020/ply/1.ply'
weights = 'weights.pth'

In [4]:
# TO VISUALIZATION
VALID_CLASS_IDS = [
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 24, 28, 33, 34, 36, 39
]

SCANNET_COLOR_MAP = {
    0: (0., 0., 0.),
    1: (174., 199., 232.),
    2: (152., 223., 138.),
    3: (31., 119., 180.),
    4: (255., 187., 120.),
    5: (188., 189., 34.),
    6: (140., 86., 75.),
    7: (255., 152., 150.),
    8: (214., 39., 40.),
    9: (197., 176., 213.),
    10: (148., 103., 189.),
    11: (196., 156., 148.),
    12: (23., 190., 207.),
    14: (247., 182., 210.),
    15: (66., 188., 102.),
    16: (219., 219., 141.),
    17: (140., 57., 197.),
    18: (202., 185., 52.),
    19: (51., 176., 203.),
    20: (200., 54., 131.),
    21: (92., 193., 61.),
    22: (78., 71., 183.),
    23: (172., 114., 82.),
    24: (255., 127., 14.),
    25: (91., 163., 138.),
    26: (153., 98., 156.),
    27: (140., 153., 101.),
    28: (158., 218., 229.),
    29: (100., 125., 154.),
    30: (178., 127., 135.),
    32: (146., 111., 194.),
    33: (44., 160., 44.),
    34: (112., 128., 144.),
    35: (96., 207., 209.),
    36: (227., 119., 194.),
    37: (213., 92., 176.),
    38: (94., 106., 211.),
    39: (82., 84., 163.),
    40: (100., 85., 144.),
}

In [5]:
def normalize_color(color: torch.Tensor, is_color_in_range_0_255: bool = False) -> torch.Tensor:
    r"""
    Convert color in range [0, 1] to [-0.5, 0.5]. If the color is in range [0,
    255], use the argument `is_color_in_range_0_255=True`.
    `color` (torch.Tensor): Nx3 color feature matrix
    `is_color_in_range_0_255` (bool): If the color is in range [0, 255] not [0, 1], normalize the color to [0, 1].
    """
    if is_color_in_range_0_255:
        color /= 255
    color -= 0.5
    return color.float()

In [6]:
def checkpoint(model, optimizer, epoch):
    mkdir_p(log_dir)
    filename = f"checkpoint_{model}.pth"
    checkpoint_file = config.log_dir + '/' + filename
    state = {
        'epoch': epoch,
        'arch': config.model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, checkpoint_file)
    logging.info(f"Checkpoint saved to {checkpoint_file}")
    if os.path.exists(f'{config.log_dir}/weights.pth'):
        os.remove(f'{config.log_dir}/weights.pth')
    os.system(f'cd {config.log_dir}; ln -s {filename} weights.pth')

In [7]:
def batch_to_device(data, device, verbose = False):
    coords, feats, labels = data
    coords = coords.to(device)
    feats = feats.to(device)
    labels = labels.to(device)

    if verbose:
        print(coords.shape)
        print(feats.shape)
        print(labels.shape)

    return coords, feats, labels

In [8]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


In [9]:
def epoch_train(model, train_dataloader, criterion, optimizer, scheduler, device):
    model.train()
    for n_iter, data in enumerate(train_dataloader):
        coords, feats, labels = batch_to_device(data, device, verbose=False)
        input = ME.SparseTensor(feats.float(), coords)
        output = model(input).F.squeeze() # SparseTensor to torch tensor 

        optimizer.zero_grad()
        loss = criterion(output, labels.float())
        loss.backward()
        optimizer.step()
        print("loss:", "{:5.5f}".format(loss.item()))

In [10]:
def epoch_test(epoch, model, train_dataloader, criterion, optimizer, scheduler, device):
    model.eval()
    for n_iter, data in enumerate(train_dataloader):
        coords, feats, labels = batch_to_device(data, device, verbose=False)
        input = ME.SparseTensor(feats.float(), coords)
        output = model(input).F.squeeze() # SparseTensor to torch tensor 

        optimizer.zero_grad()
        loss = criterion(output, labels.float())
        loss.backward()
        optimizer.step()
        print("loss:", "{:5.5f}".format(loss.item()))

        xyz = feats.cpu().numpy()
        rgb_gt = labels.cpu().numpy()
        rgb_est = output.detach().cpu().numpy()
        np.savetxt("pred/e" + str(epoch) + "_xyzrgb_est.txt", np.hstack([xyz,rgb_est]))
        np.savetxt("pred/e" + str(epoch) + "_xyzrgb_gt.txt", np.hstack([xyz,rgb_gt]))

In [11]:
def get_dataloader(video, batch_size=3, num_workers=0, shuffle=True, drop_last=False, collate=False):

    dataset = ExampleDataset(video)
    print("ExampleDataset is made")

    if not collate:
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            num_workers=0,
            drop_last=drop_last
        )
    else:
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            num_workers=0,
            drop_last=drop_last,
            # collate_fn=ME.utils.batch_sparse_collate
            collate_fn=ME.utils.SparseCollation()
        )
    print("ExampleDataset Loader is made")

    return dataloader

In [12]:
videos = [ "0051", "0052", "0053", "0054", "0055","0056", "0057", "0058", "0059", "0060"]

In [13]:
'''
def train():
    device = torch.device('cuda')
    torch.cuda.empty_cache()

    # model 
    model = MinkUNet34C(in_channels=3, out_channels=3, D=3).to(device)
    
    # optim
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-6)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
    scheduler_interval = 10

    # loss 
    criterion = nn.SmoothL1Loss()

    # a data loader must return a tuple of coords, features, and labels.
    train_dataloader = get_dataloader(batch_size=3, collate=True)

    eval_period = 25
    for epoch in tqdm(range(101)):
        #epoch_train(model, train_dataloader, criterion, optimizer, scheduler, device)
        if(epoch % eval_period == 0):
            epoch_test(epoch, model, train_dataloader, criterion, optimizer, scheduler, device)
        else:
            epoch_train(model, train_dataloader, criterion, optimizer, scheduler, device)
'''

def train():
    
    # device  
    device = torch.device('cuda')
    torch.cuda.empty_cache()

    # model 
    model = MinkUNet14(in_channels=3, out_channels=3, D=3).to(device)
    # print(model)

    # optim
    optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-6)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
    scheduler_interval = 10

    # loss 
    criterion = nn.SmoothL1Loss()

    eval_period = 25
    
    # a data loader must return a tuple of coords, features, and labels.
    #for video in videos:
    train_dataloader = get_dataloader(videos[0], batch_size=3, collate=True)
        
    for epoch in tqdm(range(40)):
        if(epoch % eval_period == 0):
            epoch_test(epoch, model, train_dataloader, criterion, optimizer, scheduler, device)
        else:
            epoch_train(model, train_dataloader, criterion, optimizer, scheduler, device)
        '''
        if epoch % 5 == 0: 
            checkpoint(model, optimizer, epoch)
        '''

In [14]:
train()

  0%|          | 0/40 [00:00<?, ?it/s]

ExampleDataset is made
ExampleDataset Loader is made
loss: 0.00263
loss: 0.00151
loss: 0.00149
loss: 0.00144
loss: 0.00122
loss: 0.00097
loss: 0.00094
loss: 0.00096
loss: 0.00093
loss: 0.00084
loss: 0.00094
loss: 0.00087
loss: 0.00082
loss: 0.00075
loss: 0.00086
loss: 0.00081
loss: 0.00064
loss: 0.00078
loss: 0.00076
loss: 0.00071
loss: 0.00071
loss: 0.00073
loss: 0.00060
loss: 0.00068
loss: 0.00066
loss: 0.00077
loss: 0.00067
loss: 0.00065
loss: 0.00063
loss: 0.00066
loss: 0.00061
loss: 0.00070
loss: 0.00068
loss: 0.00069
loss: 0.00070
loss: 0.00063
loss: 0.00066
loss: 0.00067
loss: 0.00063
loss: 0.00067
loss: 0.00061
loss: 0.00060
loss: 0.00071
loss: 0.00066
loss: 0.00059
loss: 0.00067
loss: 0.00069
loss: 0.00056
loss: 0.00058
loss: 0.00064
loss: 0.00067
loss: 0.00065
loss: 0.00055
loss: 0.00061
loss: 0.00062
loss: 0.00064
loss: 0.00055
loss: 0.00054
loss: 0.00061
loss: 0.00058
loss: 0.00056
loss: 0.00051
loss: 0.00065
loss: 0.00069
loss: 0.00061
loss: 0.00061
loss: 0.00061
loss: 0.0

  2%|▎         | 1/40 [06:36<4:17:58, 396.90s/it]

loss: 0.00022
loss: 1.41221
loss: 0.83318
loss: 0.45075
loss: 0.23292
loss: 0.12284
loss: 0.08286
loss: 0.06790
loss: 0.06173
loss: 0.04999
loss: 0.04633
loss: 0.03908
loss: 0.03181
loss: 0.03153
loss: 0.02634
loss: 0.02239
loss: 0.02326
loss: 0.01676
loss: 0.01575
loss: 0.01422
loss: 0.01427
loss: 0.01536
loss: 0.01222
loss: 0.01243
loss: 0.01181
loss: 0.01059
loss: 0.01253
loss: 0.01150
loss: 0.01164
loss: 0.00965
loss: 0.01198
loss: 0.00936
loss: 0.00775
loss: 0.00791
loss: 0.00836
loss: 0.00617
loss: 0.01034
loss: 0.00593
loss: 0.00759
loss: 0.00818
loss: 0.00644
loss: 0.00601
loss: 0.00710
loss: 0.00571
loss: 0.00563
loss: 0.00534
loss: 0.00520
loss: 0.00497
loss: 0.00602
loss: 0.00591
loss: 0.00511
loss: 0.00385
loss: 0.00575
loss: 0.00558
loss: 0.00413
loss: 0.00512
loss: 0.00381
loss: 0.00379
loss: 0.00653
loss: 0.00441
loss: 0.00350
loss: 0.00414
loss: 0.00481
loss: 0.00408
loss: 0.00438
loss: 0.00470
loss: 0.00373
loss: 0.00363
loss: 0.00381
loss: 0.00379
loss: 0.00569
loss: 

  5%|▌         | 2/40 [12:03<3:45:19, 355.79s/it]

...

 50%|█████     | 20/40 [32:59<34:02, 102.13s/it]

loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00005
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00003
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00004
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00003
loss: 0.00003
loss: 0.00005
loss: 0.00004
loss: 0.00003
loss: 0.00004
loss: 0.00003
loss: 0.00003
loss: 0.00003
loss: 0.00004
loss: 

 50%|█████     | 20/40 [34:22<34:22, 103.11s/it]

loss: 0.00003


KeyboardInterrupt: 